In [20]:
import torch
import pandas as pd
from processing import deephase_data
import model
import encoder
import esm.pretrained

In [2]:
# TODO: set hyperparams as config

my_data, categories = deephase_data()

# From ESM git repo readme
test_data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
    ("protein3",  "K A <mask> I S Q"),
]
my_esm = model.ESM()

In [13]:
x = my_data[0:4]
x_short = []
for i in x:
    short = i[1][0:60]
    x_short.append((i[0], short))

print(x_short)

batch_labels, batch_strs, batch_tokens = my_esm.batch_converter(x_short)

with torch.no_grad():
    my_results = my_esm.model(batch_tokens, repr_layers=[33], return_contacts = True)
my_token_representations = my_results["representations"][33]

[('P05453', 'AGFQPQSQGMSLNDFQKQQKQAAPKPKKTLKLVSSSGIKLANATKKVGTKPAESDKKEEE'), ('P35637', 'ASNDYTQQATQSYGAYPTQPGQGYSQQSSQPYGQQSYSGYSQSTDTSGYGQSSYSSYGQS'), ('O60885', 'CLRKKRKPQAEKVDVIAGSSKMKGFSSSESESSSESSSSDSEDSETEMAPKSKKKGHPGR'), ('G5EBV6', 'DFDVYVLAKLLGFASEELQEEIEIIRDNVTDAFEACKPLLKKLMIEGPKIDSVDPFTQLL')]


In [4]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

batch_labels, batch_strs, batch_tokens = batch_converter(test_data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

In [16]:
print(my_token_representations.size())
print(token_representations.size())

torch.Size([4, 62, 1280])
torch.Size([4, 73, 1280])


In [23]:
num_layers = 1
model_dim = 1280
num_heads = 4
ff_dim = 1280
my_encoder = encoder.Encoder(num_layers, model_dim, num_heads, ff_dim)
my_encoded = my_encoder(my_token_representations)

my_encoded.size()

torch.Size([4, 62, 1280])